In [ ]:
import math
import numpy as np
import tomlkit
import pandas as pd
import yaml
from IPython.display import display
import importlib
import copy
import awkward as ak
import seaborn as sns
sns.set_theme()
import matplotlib
import matplotlib.pyplot as plt
import os
from pathlib import Path

import study_lib

In [ ]:
importlib.reload(study_lib)
do_run = study_lib.do_run 
run_experiment = study_lib.run_experiment
config_series = study_lib.config_series

In [ ]:
base_config_yaml = f"""
candidates: 4
voters: 2000
considerations:
- Issues:
    - halfcsep: 0.
      halfvsep: 0.
      sigma: {1./math.sqrt(3.)}
      uniform: true
methods:
- Plurality:
    strat: Honest
- Plurality:
    strat: Strategic
- InstantRunoff: {{}}
- Range:
    nranks: 2
    strat: Honest
- Range:
    nranks: 2
    strat: Strategic
- STAR:
    strat: Honest
- STAR:
    strat: Strategic
    strategic_stretch_factor: 2.0
- BtrIrv:
    strat: Honest
- RP:
    strat: Honest
"""
base_config = yaml.safe_load(base_config_yaml)

In [ ]:
sfopts = dict(bbox_inches='tight', transparent=False)

In [ ]:
config = copy.deepcopy(base_config)
df = run_experiment(config_series(config, 'candidates', [2, 3, 5, 8]), trials=50000,
                      with_results=True, show_output=False)
results = df.attrs["results"]
df

In [ ]:
# rslt_5c = do_run(base_config, 10000)
(rslt_2c, rslt_3c, rslt_5c, rslt_8c) = results

In [ ]:
need_cols = ['issues', 'num_smith', 'in_smith', 'methods']  # not using cov_matrix, ideal_cand, or cand_regret for memory savings
rslt_2c = ak.from_parquet('ctr_sqz_1d_2c.parquet', columns=need_cols)
rslt_3c = ak.from_parquet('ctr_sqz_1d_3c.parquet', columns=need_cols)
rslt_5c = ak.from_parquet('ctr_sqz_1d_5c.parquet', columns=need_cols)
rslt_8c = ak.from_parquet('ctr_sqz_1d_8c.parquet', columns=need_cols)
rslt_13c = ak.from_parquet('ctr_sqz_1d_13c.parquet', columns=need_cols)

In [ ]:
rslt_5c.fields

In [ ]:
rslt_5c.methods.fields

In [ ]:
for idx, rslt in enumerate(results):
    ncand = len(rslt.issues[0, :, 0])
    print(f"resuts[{idx}]: {ncand=}")

In [ ]:
rslt=rslt_2c
iss = rslt.issues[:,:,0]
ncand = len(rslt.issues[0, :, 0])
print(f"{ncand=}")
pl_h_winner_iss = iss[np.arange(len(iss)), rslt.methods.pl_h.winner]
aprv_h_winner_iss = iss[np.arange(len(iss)), rslt.methods.aprv_h.winner]
pl_s_winner_iss = iss[np.arange(len(iss)), rslt.methods.pl_s.winner]
irv_h_winner_iss = iss[np.arange(len(iss)), rslt.methods.IRV_h.winner]
all_iss = iss[np.arange(len(iss)), np.random.randint(0,ncand,len(iss))]
max_util = rslt.issues[:, 0, 0]

In [ ]:
sns.histplot({'Random':all_iss, 'FPTP (H)':pl_h_winner_iss, 'Max utility':max_util},
             binrange=(-1,1), bins=80, stat='density')
plt.title("Two candidates (All voting methods agree)")
plt.xlabel("Issue position")
plt.savefig("plots/oneissue_twocand.png", **sfopts)

In [ ]:
rslt=rslt_5c
iss = rslt.issues[:,:,0]
ncand = len(rslt.issues[0, :, 0])
print(f"{ncand=}")
pl_h_winner_iss = iss[np.arange(len(iss)), rslt.methods.pl_h.winner]
aprv_h_winner_iss = iss[np.arange(len(iss)), rslt.methods.aprv_h.winner]
pl_s_winner_iss = iss[np.arange(len(iss)), rslt.methods.pl_s.winner]
irv_h_winner_iss = iss[np.arange(len(iss)), rslt.methods.IRV_h.winner]
all_iss = iss[np.arange(len(iss)), np.random.randint(0,ncand,len(iss))]
max_util = rslt.issues[:, 0, 0]
sns.histplot({'Random':all_iss, 'FPTP (H)':pl_h_winner_iss, 'Max utility':max_util},
             binrange=(-1,1), bins=80, stat='density')
plt.title("Five candidates")
plt.xlabel("Issue position")
plt.savefig("plots/oneissue_5cand_fptp_hist.png", **sfopts)

In [ ]:
sns.histplot({'RCV (honest)':irv_h_winner_iss, 'FPTP (Strat)':pl_s_winner_iss, 'Max utility':max_util},
             binrange=(-1,1), bins=80, stat='density')
plt.title("Five candidates")
plt.xlabel("Issue position")
plt.xlim([-1,1])
plt.savefig("plots/oneissue_5cand_fptps_irv_hist.png", **sfopts)

In [ ]:
ak.count_nonzero(rslt_5c.in_smith[np.arange(len(iss)), rslt_5c.methods.aprv_h.winner])

In [ ]:
ak.count_nonzero(rslt_5c.num_smith < 3)

In [ ]:
ak.count_nonzero(rslt_5c.in_smith[np.arange(len(iss)), rslt_5c.methods["BTR-IRV_h"].winner]) 

In [ ]:
rslt_5c.methods.fields

In [ ]:
ak.count_nonzero(rslt_5c.in_smith[np.arange(len(iss)), rslt_5c.methods["rp_h"].winner])

In [ ]:
def hist_1d_issue(rslt, method, ax=None, nbins=40, title="", y2lims=None, simple_y2lbls=True):
    ncand = len(rslt.issues[0, :, 0])
    if ax is None:
        _, ax = plt.subplots(1, 1)
    iss = rslt.issues[:,:,0]
    winner_iss = iss[np.arange(len(iss)), rslt.methods[method].winner]
    best_iss = rslt.issues[:, 0, 0]
    winner_hist, bin_edges = np.histogram(winner_iss, bins=nbins, range=(-1,1))
    best_hist, _ = np.histogram(best_iss, bins=nbins, range=(-1,1))
    bin_width = bin_edges[1] - bin_edges[0]
    bin_centers = bin_edges[:-1] + bin_width / 2
    winner_dist_ratio = winner_hist / best_hist
    best_theory = (1. - np.abs(bin_edges))**ncand
    best_theory = np.abs(best_theory[1:] - best_theory[:-1]) / 2 / bin_width
    sns.histplot(winner_iss, binrange=(-1,1), bins=nbins, stat='density', ax=ax, label="winner dist'n")
    ax.plot(bin_centers, best_theory, 'P', markersize=3, c='C1', label="theory, ∞ voters")
    ax.set_xlim([-1, 1])
    ax2 = ax.twinx()
    ax2.semilogy(bin_centers, winner_dist_ratio, '.-', label="winners / max_util", c="C5")
    ax2.set_ylabel("Ratio winner / best candidate")
    if simple_y2lbls:
        ax2.yaxis.set_minor_formatter(matplotlib.ticker.ScalarFormatter())
        ax2.yaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())
    ax2.tick_params(axis='y', which="both", colors='C5')
    #ax2.yaxis.label.set_color("C4")
    if y2lims:
        ax2.set_ylim(y2lims)
    ax.grid(which="major", axis="y")  # is this a bug?
    ax2.grid(which="minor", axis="y")  # or is it this one?
    ax.set_xlabel("Issue position")
    if title:
        plt.title(title)
    h1, l1 = ax.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax.legend(h1+h2, l1+l2, loc="upper right")

In [ ]:
hist_1d_issue(rslt_5c, 'rp_h', nbins=40, title='5 Candidates, Ranked Pairs', y2lims=(.5,2.0))

In [ ]:
hist_1d_issue(rslt_2c, 'rp_h', nbins=40, title='2 Candidates, Any method', y2lims=(.5,2.0))
plt.savefig("plots/oneissue_twocand_ratio.png", **sfopts)

In [ ]:
rp_h_2c_misses = rslt_2c[rslt_2c.methods.rp_h.winner != rslt_2c.ideal_cand]
print(f"{len(rp_h_2c_misses)=}")
ak.to_list(rp_h_2c_misses[0])

In [ ]:
ak.to_list(rp_h_2c_misses[3])

In [ ]:
def make_figs(results_, ncands_, column, title, filename, y2lims=None, simple_y2lbls=True):
    for rslt, ncand in zip(results_, ncands_):
        hist_1d_issue(rslt, column, nbins=40, title=title.format(ncand), y2lims=y2lims, simple_y2lbls=simple_y2lbls)
        fn = Path(filename.format(ncand))
        os.makedirs(fn.parent, exist_ok=True)
        plt.savefig(fn, **sfopts)

In [ ]:
results = [rslt_3c, rslt_5c, rslt_8c, rslt_13c]
ncands_list = [3, 5, 8, 13]
make_figs(results, ncands_list, 'rp_h', "{} Candidates, Ranked Pairs", "plots/oneissue/rp/{:02} Cands.png")

In [ ]:
make_figs(results, ncands_list, 'star_6_h', "{} Candidates, STAR", "plots/oneissue/star/{:02} Cands.png")

In [ ]:
make_figs(results, ncands_list, 'IRV_h', "{} Candidates, RCV", "plots/oneissue/irv/{:02} Cands.png")

In [ ]:
make_figs(results, ncands_list, 'pl_s', "{} Candidates, FPTP Strategic", "plots/oneissue/fptps/{:02} Cands.png", simple_y2lbls=False)
make_figs(results, ncands_list, 'pl_h', "{} Candidates, FPTP Honest", "plots/oneissue/fptph/{:02} Cands.png", simple_y2lbls=False)

In [ ]:
make_figs(results, ncands_list, 'star_6_s', "{} Candidates, STAR Strategic", "plots/oneissue/stars/{:02} Cands.png")

In [ ]:
# hist_1d_issue(rslt_3c, 'aprv_h', nbins=40, title='3 Candidates, Approval (Honest)') #, y2lims=(.9,1.1))
# plt.savefig("plots/oneissue_aprv_3c.png")
make_figs(results, ncands_list, 'aprv_h', "{} Candidates, Approval Honest", "plots/oneissue/aprv/{:02} Cands.png")
make_figs(results, ncands_list, 'aprv_s', "{} Candidates, Approval Strategic", "plots/oneissue/aprvs/{:02} Cands.png")

## Prop 131

Open primary, FPTP. Top 4 candidates go to IRV.

In [ ]:
p131_config_yaml = f"""
candidates: 4
primary_candidates: 12
primary_method:
    PluralityTopN: {{}}
voters: 3001
considerations:
- Issues:
    - halfcsep: 0.
      halfvsep: 0.
      sigma: {1./math.sqrt(3.)}
      uniform: true
methods:
- InstantRunoff: {{}}
"""
p131_config = yaml.safe_load(p131_config_yaml)

In [ ]:
rslt_p131 = do_run(p131_config, 100000)

In [ ]:
rslt=rslt_p131
iss = rslt.issues[:,:,0]
ncand = len(rslt.issues[0, :, 0])
print(f"{ncand=}")
irv_h_winner_iss = iss[np.arange(len(iss)), rslt.methods.IRV_h.winner]
finalist_iss = ak.flatten(iss)

In [ ]:
sns.histplot({'RCV winner':irv_h_winner_iss, 'Finalists':finalist_iss},
             binrange=(-1,1), bins=80, stat='density', common_norm=False)
plt.title("Prop 131 simulation, 12 primary candidates")
plt.xlabel("Issue position")
plt.xlim(-1, 1)
plt.savefig("plots/prop131.png", **sfopts)